# ResponseGetData

> preferred response class for all API requests

In [ ]:
# | default_exp ResponseGetData


In [ ]:
# | hide
from nbdev.showdoc import show_doc
from fastcore.test import test_eq


In [ ]:
# | export
import requests
import asyncio
import aiohttp

from dataclasses import dataclass, field
from typing import Union, Optional

from fastcore.utils import patch_to


All API routes should return an instance of `ResponseGetData`.

In [ ]:
# | export

@dataclass
class ResponseGetData:
    """preferred response class for all API routes"""
    status: int
    response: Union[list, dict, str]
    is_success: bool
    auth_header: Optional[dict] = field(default = None, repr = False)


#### Sample Implementation of ResponseGetData

In [ ]:
rgd = ResponseGetData(status=200, response="test", is_success=True)
rgd


ResponseGetData(status=200, response='test', is_success=True, auth_header=None)

In [ ]:
# | hide
test_eq(rgd.is_success, True)


# Generate ResponseGetData with classmethods

Users can generate an instance of the `ResponseGetData` class from two private classmethods (see below).

This class currently supports implementations of two API request libraries, `requests.request` (synchronous) and `aiohttp.ClientRequest` (asynchronous).

In [ ]:
# | export
@patch_to(ResponseGetData, cls_method=True)
def _from_requests_response(cls, res: requests.Response,  # requests response object
                            auth_header: Optional[dict] = None # auth header used to authenticate request
                            ) -> ResponseGetData:
    """returns ResponseGetData"""

    # JSON responses
    if res.ok and "application/json" in res.headers.get("Content-Type", {}):
        return ResponseGetData(
            status=res.status_code,
            response=res.json(),
            is_success=True,
            auth_header=auth_header
        )

    # default text responses
    elif res.ok:
        return ResponseGetData(
            status=res.status_code,
            response=res.text,
            is_success=True,
            auth_header=auth_header
        )

    # errors
    return ResponseGetData(
        status=res.status_code,
        response=res.reason,
        is_success=False,
        auth_header=auth_header
    )


In [ ]:
show_doc(ResponseGetData._from_requests_response)


---

[source](https://github.com/jaewilson07/nbdev_domo/blob/main/nbdev_domo/ResponseGetData.py#L29){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_requests_response

>      ResponseGetData._from_requests_response (res:requests.models.Response,
>                                               auth_header:Optional[dict]=None)

returns ResponseGetData

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| res | Response |  | requests response object |
| auth_header | Optional | None | auth header used to authenticate request |
| **Returns** | **ResponseGetData** |  |  |

#### sample implementation of _from_requests_response

In [ ]:
test_url = 'http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita'

res = requests.request(method='GET', url=test_url)

test_res = ResponseGetData._from_requests_response(res, auth_header = {'x-domo-authentication': 'test123'})
test_res.__dict__.keys()


dict_keys(['status', 'response', 'is_success', 'auth_header'])

In [ ]:
# | hide
test_eq(isinstance(test_res, ResponseGetData), True)


In [ ]:
# | export
@patch_to(ResponseGetData, cls_method=True)
async def _from_aiohttp_response(cls, res: aiohttp.ClientResponse,  # requests response object
                                 auth_header: Optional[dict] = None, # auth header used to authenticate request
                                 ) -> ResponseGetData:

    """async method returns ResponseGetData"""

    if res.ok and "application/json" in res.headers.get("Content-Type", {}):
        try:
            return cls(
                status=res.status, response=await res.json(), is_success=True, auth_header=auth_header
            )

        # handle if unable to decode json()
        except asyncio.TimeoutError as e:
            print(e)

            return cls(
                status=res.status, response=await res.json(content_type=None), is_success=True, auth_header=auth_header
            )

        except Exception as e:
            print(e)

            data = await res.read()
            return cls(
                status=res.status, response=data.decode(), is_success=True, auth_header=auth_header
            )

        # response is text
    elif res.ok:
        return cls(
            status=res.status, response=await res.text(), is_success=True, auth_header=auth_header
        )

    # response is error
    else:
        return cls(status=res.status, response=str(res.reason), is_success=False, auth_header=auth_header)


In [ ]:
show_doc(ResponseGetData._from_aiohttp_response)


---

[source](https://github.com/jaewilson07/nbdev_domo/blob/main/nbdev_domo/ResponseGetData.py#L63){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_aiohttp_response

>      ResponseGetData._from_aiohttp_response
>                                              (res:aiohttp.client_reqrep.Client
>                                              Response,
>                                              auth_header:Optional[dict]=None)

async method returns ResponseGetData

#### sample implementation of _from_aiohttp_response.
Notice the use of async / await

In [ ]:
test_url = 'http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita'

timeout = aiohttp.ClientTimeout(total=10)
session = aiohttp.ClientSession(timeout=timeout)

try:
    res = await session.get(url=test_url)

    test_res = await ResponseGetData._from_aiohttp_response(res)
    print(test_res.__dict__.keys())
except TimeoutError as e:
    # sometimes aiohttp errors out for unexpected reasons.
    print(e)

finally:
    await session.close()


dict_keys(['status', 'response', 'is_success', 'auth_header'])


In [ ]:
# | hide
test_eq(isinstance(test_res, ResponseGetData), True)

rgd = ResponseGetData(status=200, response="test", is_success=True)

test_eq(rgd.is_success, True)

rgd


ResponseGetData(status=200, response='test', is_success=True, auth_header=None)

In [ ]:
# | hide
import nbdev
nbdev.nbdev_export()
